# Analyzing No mans Sky EXML files

In [1]:
# If you are new to Pyton, and just want to run this file, install conda
# https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html

from lxml import etree
import random as random
import pickle
import networkx as nx
import matplotlib.pyplot as plt
import tabulate as tb

## Language files

In [62]:
# usful functions
def get_value_for_name(name, elements):
    '''
    takes avalue of a name field and returns the value from a iterable collection of Elements
    '''
    for e in elements:
        if e.get('name') == name:
            return e.get('value')
    
#Look up text IDs and their files
def locate_id(Id):
    print("Id: "+ Id) 
    print("Text: " + id_text[Id])
    print("File: " + id_file[Id])

def is_in_string(substring, fullstring):
    if substring in fullstring:
        return True
    else:
        return False

def get_tag(tag,string):
    open_tag = tag
    close_tag = "<>"
    while len(string) > 0 or len(string) < 0:
        sub_idx =string.find(open_tag)
        #print("open_idx: ", str(sub_idx))
        string = string[sub_idx + len(open_tag):]
        #print(string)
        sub_idx = string.find(close_tag)
        #print("close_idx: ", str(sub_idx))
        content = string[: sub_idx ]
        return content

def locate_word(word):
    locs = {}
    for k,v in id_text.items():
        if is_in_string(word,v):
            locs[k]= v
    return locs

def find_tags(tag) :
    audo_strings = []
    for k, v in id_text.items():
        if is_in_string(tag,v):
            #print("-------------")
            #print(v)
            text = get_tag(tag,v)
            if text is not None:
                audo_strings.append(text)
    return audo_strings

def get_elem_name(e,lvl):
    n = e.attrib.get('name')
    if n == None:
        n = "NO_NAME"
    v = e.attrib.get('value')
    if v == None:
        v = "NO_VALUE"
    return n + ':' + v   + ":lvl:"+str(lvl)


def element_name_value(elem, from_node, lvl):
    lvl = lvl + 1
    #print("level: " + str(lvl))
    local_pair = []
    to_node = get_elem_name(elem,lvl)
    link = [from_node,to_node]
    local_pair.append([from_node,to_node])
    
    from_node = get_elem_name(elem,lvl)
    for e in elem.iterchildren() :
        local_pair.extend(element_name_value(e,from_node,lvl))
    return local_pair

def get_graph(root_node):
    lvl = 0
    node_pairs = [] 
    root_node_name = get_elem_name(root_node,lvl)
    pairs = element_name_value(root_node, root_node_name, lvl)
    node_pairs.extend(pairs)
    return node_pairs
 

#set(find_tags("<IMG>"))
    
#locate_word("4925C")
#locateId('MISSION_OBJ_COLLECT')    

In [101]:
# Load langage files and put the in dictionaries
# There is also 'NMS_LOC1_USENGLISH.EXML, but we will ignore that one, asit has identical keys
files =['NMS_LOC1_ENGLISH.EXML',
        'NMS_LOC4_ENGLISH.EXML',
        'NMS_LOC5_ENGLISH.EXML',
        'NMS_LOC6_USENGLISH.EXML',
        'NMS_UPDATE3_ENGLISH.EXML'
       ]
#string id : file it comes from
id_file = {}
# stringd id : the actual text
id_text = {}

for f in files:
    tree = etree.parse("language/" + f)
    root = tree.getroot()
    
    for child in root[0]:
        x = child[0].get('value')
        y = child[1][0].get('value')
        id_file[x] = f
        id_text[x] = y

In [4]:
# Processing can take a while. After the first run, you can save the dictionaries to disk, and just loead next tiem instead of parsing
#pickle.dump(id_text,save( "Id_to_text_dictionary.p", "wb" ))



In [5]:
#lets print a random one, just becasue we can
random_id = random.choice(list(id_text.keys()))
print(random_id)
print(id_text[random_id])
print(id_file[random_id])


UI_PHOTO_BLD_OBJ_MSG1
Take a photo of %BUILDING%
Activate photo mode with <IMG>QUICK_MENU<>
NMS_UPDATE3_ENGLISH.EXML


### lets pick out special tags that occur in dialogues

In [6]:
#TODO add auto tag extraction
# IMG tags :
IMG_tags = list(set(find_tags("<IMG>")))
print(IMG_tags)

['LANDSHIP', 'EXPAND', 'GALAXY_GESTURE', 'NOTICK', 'EXOBOOST', 'RUN', 'VR_TELEPORT', 'SHIPSCAN', 'BULLETPOINT', 'DISNETRETRY', 'ALTATTACK', 'INVBACKPACK', 'TRANSFER_SHIP', 'MOVE', 'OPTIONS', 'GALAXYWAYPT', 'ATTACK', 'TRADEABLE', 'HOTKEY1', 'PLAYER_RIGHT', 'SIZETERRAIN_DN', 'SHOW_OBJECTIVE', 'SHIPCYCLEWEAPON', 'PIN', 'UI_SPLITUP', 'GALAXYWARP', 'MISSION_TIP', 'CC_EXIT', 'PULSEJUMP', 'TELEPORT_AIM', 'BOOST', 'DANGER', 'QUICK_MENU', 'HARVESTPLANT', 'PLACE_RACE_NODE', 'INTERACT_LEFT', 'BUILD', 'BUILD_PLACE', 'SCAN', 'COMMODITY', 'TECHMENU', 'GMSEL', 'DESELECT', 'Tradeable', 'QUICK_LEFT', 'FIRE', 'FE_BACK', 'CC_CAMROT', 'BUILD_SELECT', 'EDITTERRAIN', 'GAMERCARD', 'BUILD_ROTATE', 'FE_ALTSELECT', 'INTERACT', 'UNITS', 'MELEE', 'VR_SNAPLEFT', 'CLOCK', 'SURVEYFILTERL', 'ROTTERRAINLEFT', 'THRUST', 'OPTIONS>', 'PHOTO_CAM', 'LOOK', 'UI_LEFT', 'DIAMONDBULLET', 'GMBACK', 'CYCLEWEAPON', 'BUILD_CAM', 'TECHNOLOGY', 'CYCLEALTWEAPON', 'FRIGMINE', 'GALAXYSCAN', 'GALAXYSCANNXT', 'ROLLLEFT', 'SIZETERRAIN_UP'

In [7]:
# AUDIO tags :
AUDIO_tags = list(set(find_tags("<AUDIO>")))
print(AUDIO_tags )

['Portal_Jump_Story_End', 'WPN_PL_PISTOL', 'WPN_PL_TAKEOUT', 'TXT_StaticNoise_Start', 'UI_Standing_Increased', 'MUS_StoryMode_MusicCue_10_Lp', 'UI_Nanites_Received', 'UI_Destination_Reached', 'WARRIORANGRY', 'CrystalAltar_Collect', 'WPN_PL_MELEE', 'MUS_StoryMode_MusicCue_10_Lp_Stop', 'Communicator_Open', 'TXT_RadioNoise', 'UI_Record_Uploaded', 'UI_NoWhereToLand']


# Examining Missions

In [10]:
#load the files
mission_roots={}

# There are Property elements, that have different attributes
npc_mission_tree = etree.parse("missions/NPCMISSIONTABLE.EXML")
npc_mission_root = npc_mission_tree.getroot()
mission_roots["NPCMISSIONTABLE.EXML"] = npc_mission_root 

In [11]:
tutorial_mission_tree = etree.parse("missions/TUTORIALMISSIONTABLE.EXML")
tutorial_mission_root = tutorial_mission_tree.getroot()
mission_roots["TUTORIALMISSIONTABLE.EXML"] = tutorial_mission_root

In [13]:
wiki_mission_tree = etree.parse("missions/WIKIMISSIONTABLE.EXML")
wiki_mission_root = wiki_mission_tree.getroot()
mission_roots["WIKIMISSIONTABLE.EXML"] = wiki_mission_root

In [15]:
core_mission_tree = etree.parse("missions/COREMISSIONTABLE.EXML")
core_mission_root = core_mission_tree.getroot()
mission_roots["COREMISSIONTABLE.EXML"] = core_mission_root

In [13]:
#help(etree.ElementTree)

In [17]:
#lets get all possible properties.
# For each property we also list all unique vslues ocuring in the table

#here it means all possible 'name' attributes of Property elements
all_names_of_properties = set()
name_value = {}

i =0
for filename,root in mission_roots.items() :
    for p in root.iter():
        name = p.get("name")
        all_names_of_properties.add(name)
        #add the current list, and then get the unique elements by turning it to set, and then back to list
        val = p.get("value")
        if type(name) is not type(None):
            if type(val) is not type(None):
                if name in name_value.keys():
                    name_value[name].append(val)
                    name_value[name] = list(set(name_value[name]))
                    #print(name_value[name])
                    #cur = name_value[name]
                    #cur.append(val)
                    #new = list(set(cur))
                    #new = cur
                    #name_value[name] = new
                else:
                    name_value[name] = [val]

#print(list(all_names_of_properties))
output = tb.tabulate(name_value.items(), headers=["Property name", "Unique values"],tablefmt = "mediawiki")
#print(output)
f = open("MissionVariablesAndvalues_mediawiki.txt", "w")
f.write(output)
f.close()

In [51]:
#Get the strings that belong to a speciffic variables 

def find_strings_for(key_to_look_for):
    results = ""
    for item in name_value[key_to_look_for]:
        if item in id_text:
            #print("Key: " + item + "Text: " + id_text[item])
            results += "Key: " + item + "Text: " + id_text[item] + "\n"
        else :
            #print("Key: " + item + " is not present in the language file")
            if is_in_string("%d",item):
                #print("\t" + item + " is a numbered variable.")
                more_numbers = True
                num = 1
                while more_numbers :
                    numbered_key = item.replace("%d",str(num))
                    if numbered_key in id_text:
                        #print("\t Numbered key: " + numbered_key + " Text: " + id_text[numbered_key])
                        results += "\t Numbered key: " + numbered_key + " Text: " + id_text[numbered_key] + "\n"
                        num = num + 1
                    else:
                        more_numbers = False
    return results

                    
            
#print(find_strings_for("Format"))
#print(find_strings_for("Name"))
#print(find_strings_for("OSDMessage"))
#print(find_strings_for("Message"))

In [44]:
# Get all scan events :
ScanEvents


for filename,root in mission_roots.items() :
    for p in root.iter():
        name = p.get("name")
        if name = "ScanEvents"
        all_names_of_properties.add(name)
        #add the current list, and then get the unique elements by turning it to set, and then back to list
        val = p.get("value")
        if type(name) is not type(None):
            if type(val) is not type(None):
                if name in name_value.keys():
                    name_value[name].append(val)
                    name_value[name] = list(set(name_value[name]))
                    #print(name_value[name])
                    #cur = name_value[name]
                    #cur.append(val)
                    #new = list(set(cur))
                    #new = cur
                    #name_value[name] = new
                else:
                    name_value[name] = [val]

#print(list(all_names_of_properties))
output = tb.tabulate(name_value.items(), headers=["Property name", "Unique values"],tablefmt = "mediawiki")
#print(output)
f = open("MissionVariablesAndvalues_mediawiki.txt", "w")
f.write(output)
f.close()


In [103]:
# lets get all available missions
missions = []
missions_name_element = {}
for filename,root in mission_roots.items() :
    for m in root.iter('Property'):
        #missions are a sequence of events
        if m.get('value') == "GcGenericMissionSequence.xml":
            missions.append(m)
            #get all children Property elements, belonging to this element.
            elements = m.iter()
            _text = get_value_for_name('MissionID',elements)
            missions_name_element[_text] = m
            print("MissionID: " + _text )
            mission_objective = get_value_for_name('MissionObjective',elements)
            if mission_objective in id_text:
                print("Mission description: " + id_text[mission_objective])
#missions

MissionID: BOUNTY_EASY
Mission description: Hunt Low Level Pirates
MissionID: BOUNTY_MED
Mission description: Hunt Pirates
MissionID: BOUNTY_HARD
Mission description: Hunt Dangerous Pirates
MissionID: SCAN_CRE
Mission description: Scan Creatures
MissionID: SCAN_MIN
Mission description: Scan Minerals
MissionID: SCAN_TREE
Mission description: Scan Flora
MissionID: KILL_ROBOTS
Mission description: Eliminate Sentinels
MissionID: KILL_ROBOT_MED
Mission description: Eliminate Advanced Sentinels
MissionID: KILL_ROBOT_HARD
Mission description: Eliminate Advanced Sentinels
MissionID: KILL_CREATURES
Mission description: Exterminate Planetary Creatures
MissionID: KILL_PREDATORS
Mission description: Hunt Predatory Creatures
MissionID: DELIVER
Mission description: Deliver an Item
MissionID: DELIVER_HARD
Mission description: Deliver an Item
MissionID: COLLECT1
Mission description: Collect an Item
MissionID: COLLECT2
Mission description: Collect an Item
MissionID: COLLECT3
Mission description: Collec

In [122]:
#get_graph()
#help(etree._Element)
#mission_element = missions_name_element['STATION_MYSTERY']
mission_element  = npc_mission_root.xpath("//Property[@value='GcGenericMissionSequence.xml']")
output = etree.tostring(mission_element, encoding='unicode', method='XML',  pretty_print=True)
#mission_element.write('output.xml', pretty_print=True)
f = open("test.EXML", "w")
f.write(output)
f.close()

In [125]:
len(mission_element)

In [131]:
id_text["WAR_DRINK"]


'drink'

In [118]:
#fins elements of a sectain type
"GcMissionSequencePirates.xml"
#npc_mission_root.findall("Property[@name='GcMissionSequencePirates.xml']")
npc_mission_root.xpath("//Property[@value='GcMissionSequencePirates.xml']")

[<Element Property at 0x220fcd29540>, <Element Property at 0x220fcd2bb00>]

In [59]:
#all mission seqences
mission_sequences = []
mission_sequence_names = []
for filename,root in mission_roots.items() :
    for m in npc_mission_root[0]:
        #missions are a sequence of events
        for e in m.iter():

            if str(e.get('value')).find("MissionSequence") > -1 :
                val = e.get('value')
                mission_sequences.append(e)
                mission_sequence_names.append(val)
            #get all children Property elements, belonging to this element.
    n_v = []
    output = ""
    for m in mission_sequences:
        output = output + "\n"+ "=== " + m.get('value') + " ===\n"

        for e in m.iterchildren():
            name = str(e.get('name'))
            value = str(e.get('value'))
            n_v.append([name,value])
        output = output + tb.tabulate(n_v, headers=["Property name", "Value"],tablefmt = "mediawiki")
        n_v = []
    
f = open("all_MissionSequences_mediawiki.txt", "w")
f.write(output)
f.close()
    
#set(mission_sequence_names)

In [60]:
# Find unique mission sequences
unique_ms = []
for i in list(set(mission_sequence_names)):
    unique_ms.append([i])
unique_ms
output = tb.tabulate(unique_ms, headers=["Types of MissionSequence"],tablefmt = "mediawiki" )
f = open("Unique_MissionSequences_mediawiki.txt", "w")
f.write(output)
f.close()

# Graphing the structure of missions file

In [20]:
#Make a graph of the mission table structure
#Get every name, value pair, and nest them somehow

#element_name_value(npc_mission_root,"ROOT")        
        
    

np = get_graph(npc_mission_root[0])
#np = element_name_value(npc_mission_tree,"ROOT")
#np = element_name_value(npc_mission_root)
#for child in npc_mission_root[0]:
#        print(child)
#        if len(child):
#            print(get_elem_name(child))

#for element in npc_mission_root.iter():
#    print(get_elem_name(element))
##
#tree = etree.tostring(npc_mission_root, pretty_print=True)
#print(tree)



In [21]:
graph = nx.Graph(np)

pos = nx.kamada_kawai_layout(graph, scale = 20)

In [ ]:
pickle.dump( pos, open( "mission_graph_positions.p", "wb" ) )
#pos = pickle.load( open( "mission_graph_positions.p", "rb" ) )


In [ ]:
options = {
    "font_size": 3,
    "node_size": 5,
    "node_color": "blue",
    "edgecolors": "black",
    "linewidths": 0.2,
    "width": 0.2,
}


# Set margins for the axes so that nodes aren't clipped
fig = plt.figure(num=None, figsize=(10, 10), dpi=360)
ax = plt.gca()
ax.margins(0.20)
plt.axis("off")
nx.draw_networkx(graph, pos, **options)
plt.show()

In [ ]:
# Import packages for data visualization
import plotly.offline as py
import chart_studio.plotly as py_online
import plotly.graph_objects as go
import chart_studio

In [ ]:
py.init_notebook_mode()

def make_edge(x, y, text, width):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = (text),
                       mode      = 'lines')

def make_node(x_cor, y_cor, text):
    return go.Scatter(  x         = [x_cor],
                        y         = [y_cor],
                        text      = ([text]),
                        textposition = "top center",
                        textfont_size = 4,
                        mode      = 'markers',
                        hoverinfo = 'text',
                        marker    = dict(color = ['cornflowerblue'],
                                         size  = [2],
                                         line  = dict(width=0.3, color='#888')))
    
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in graph.edges():
    char_1 = edge[0]
    char_2 = edge[1]
    x0, y0 = pos[char_1]
    x1, y1 = pos[char_2]
    text   = ""

    trace  = make_edge([x0, x1, None], [y0, y1, None], text, width = 0.2)
    edge_trace.append(trace)

    
'''
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 5,
                        mode      = 'markers',
                        hoverinfo = 'text',
                        marker    = dict(color = ["black"],
                                         size  = [5],
                                         line  = dict(width=1, color='#888')
                                        )
                       )
    

# For each node in midsummer, get the position and size and add to the node_trace
nodes= []
x_list = []
y_list = []
text_list = []

for node in graph.nodes():
    x, y = pos[node]
    x_list.append(x)
    y_list.append(y)
    text_list.append(node)
node_trace['x'] =  x_list
node_trace['y'] = y_list
node_trace['text'] = text_list
'''
node_trace = []
for node in graph.nodes():
    #print(x,y)
    #print(node)
    x, y = pos[node]
    new_node = make_node(x,y,node)
    node_trace.append(new_node)
  
    
# Customize layout
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)

In [ ]:
print(node_trace[2])
node_trace[3]['text'][0].find("GcGenericMissionSequence")

In [ ]:
#Modify the cener nodes
#NO_NAME:GcGenericMissionSequence.xml
#Missions:NO_VALUE
for node in node_trace :
#    print(str(node['text'][0]))
    if node['text'][0].find("Missions:NO_VALUE") > -1 :
        node['marker'] = dict(color = ["green"],size  = [10],line  = dict(width=1, color='#888'))
    if node['text'][0].find("GcGenericMissionSequence") > -1:
        node['marker'] = dict(color = ["red"],size  = [10],line  = dict(width=1, color='#888'))
    

#node_trace[0]

In [ ]:
# Create figure
fig = go.Figure(layout = layout)
# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)
    
# Add node trace
for trace in node_trace:
    fig.add_trace(trace)

# Remove legend
fig.update_layout(showlegend = False)
# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)
# Show figure
fig.show()

In [ ]:
fig.write_html("MissionBoard.html")
fig.savefig("NPCMISSIONTABLE_structure.png",dpi = "figure" )

In [ ]:
# put it onlne
py_online.iplot(fig, filename = "MissionBoardStructure")

In [ ]:
 for child in npc_mission_root.iterchildren():
        print("--------1--------------")
        print(get_elem_name(child))
        print("----------------------")
        for ch in child.iterchildren():
            print("----2----")
            print(get_elem_name(ch))
            for c in ch.iterchildren():
                print("--3--")
                print(get_elem_name(c))
                for a in c.iterchildren():
                    print("-4-")
                    print(get_elem_name(a))
                    for b in a.iterchildren():
                        print("-5-")
                        print(get_elem_name(b))
                        for d in b.iterchildren():
                            print("-6-")
                            print(get_elem_name(d))

                    
        

In [ ]:
random.choice(np)

#np

In [ ]:
#lets examine a single missions
m = random.choice(missions)

for p in m:
    print("----- Property")
    print("\t" + str(p.attrib))
    for q in p:
        print("---------------- SubProperty")
        print("\t\t" + str(q.attrib))
        

In [ ]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(open("test_story/METADATA/SIMULATION/MISSIONS/NPCMISSIONTABLE.EXML"), 'xml')
print(bs.prettify())